# Extending PointNet networks and simulation with external and customized models.

## Example: Using Built-In NEURONAL Models

PointNet utlizes the NEST simulator for running point-neuron simulations, which includes many different varieties and types of neuronal models the majority of which can readily used during simulation in BMTK. An very useful practice is to take an existing network model built on model of a cell and replace it with a different cell model that more closely fits the needs of our research. With BMTK and SONATA this can be easily done, often without any programming, and even done on the fly before any given simulation.

A list of available models built into the NEST simulator can be found [here](https://nest-simulator.readthedocs.io/en/stable/models/index.html).

Most of the spiking models will work out-of-the-box in BMTK, readily swapped in and out of existing simulations, and you can use multiple models in the same network. (*We will ignore rates-based models which can only tentively work with BMTK. And for devices like genators and recorders PointNet uses a separate way of inserting them into the simulation*). To define what model will be used with a specific subset of cells and their properties, SONATA has two defined reserved attributes `model_template` and `dynamics_params` which we will take advantage of in this example.

### Setting the model type in our network.

The default way to set the model type for subset of neurons (which may be just a single neuron) that all use the same built-in NEST model type is to set the the **model_type** attributes to a value `nest:<nest-model-name>` where `<nest-model-name>` is the name of one of the spiking [NEST cell models](https://nest-simulator.readthedocs.io/en/stable/models/index.html). However, different models will have different parameters. There are multiple ways to set model parameters in BMTK, but the one that is most flexible and will make it easier to switch out different models while testing and optimizing our network is to set the **dynamics_params** to an easy to edit json file.

In the below example (*build_network.built_in.py*) will built a simple 100 cell network with synaptic stimuli. Initialally we will use [izhikevich](https://nest-simulator.readthedocs.io/en/stable/models/izhikevich.html) models using default params (**hint**: if the dynamics_params .json file is just an empty json file, then the cells will be loaded with the default params. This is a useful starting point). But you can change the **nest_model** (and optionally the **dynamics_params**) variable below to build the same network using different model types

In [2]:
nest_model = 'nest:izhikevich'
# nest_model = 'nest:iaf_psc_delta'
# nest_model = 'nest:aeif_cond_alpha'
# nest_model = 'nest:glif_psc'

dynamics_params = 'custom_model_params.default.json'
# dynamics_params = 'custom_model_params.izhikevich.json'
# dynamics_params = 'custom_model_params.aeif_cond_alpha.json'
# dynamics_params = 'custom_model_params.glif_psc.json'

In [3]:
import numpy as np
from bmtk.builder import NetworkBuilder


net = NetworkBuilder('net')
net.add_nodes(
    N=100,
    model_type='point_neuron',
    model_template=nest_model,
    dynamics_params=dynamics_params
)

net.add_edges(
    source=net.nodes(), target=net.nodes(),
    connection_rule=1,
    syn_weight=2.0,
    delay=1.5,
    dynamics_params='ExcToInh.json',
    model_template='static_synapse'
)

net.build()
net.save(output_dir='network_built_in')


virt_exc = NetworkBuilder('virt_exc')
virt_exc.add_nodes(
    N=10,
    model_type='virtual'
)

virt_exc.add_edges(
    target=net.nodes(),
    connection_rule=lambda *_: np.random.randint(0, 10),
    syn_weight=2.0,
    delay=1.0,
    dynamics_params='ExcToInh.json',
    model_template='static_synapse'
)

virt_exc.build()
virt_exc.save(output_dir='network_built_in')

If you have access to an existing model but not the build-script, or if rebuilding the network is too time consuming and/or expensive, then another option to swap out different models is to open the *node_types.csv* in an text editor (especially ones that support csv forats like VSCode or Atom), update the **model_template** and **dynamics_params** attributes, then rerun the simulation.

In [4]:
import pandas as pd

pd.read_csv('network_built_in/net_node_types.csv', sep=' ')

,node_type_id,model_template,dynamics_params,model_type
0,100,nest:izhikevich,custom_model_params.default.json,point_neuron


We encourage you to try running the network yourself by either running in the command line or un
```
$ python run_pointnet.built_in.py config.built_in.json
```
or with the below cell, trying it with different cell models. 

In [5]:
from bmtk.simulator import pointnet

configure = pointnet.Config.from_json('config.built_in.json')
configure.build_env()

network = pointnet.PointNetwork.from_config(configure)
sim = pointnet.PointSimulator.from_config(configure, network)
sim.run()


              -- N E S T --
  Copyright (C) 2004 The NEST Initiative

 Version: 3.6.0
 Built: Sep 28 2023 11:52:40

 This program is provided AS IS and comes with
 NO WARRANTY. See the file LICENSE for details.

 Problems or suggestions?
   Visit https://www.nest-simulator.org

 Type 'nest.help()' to find out more about NEST.

2024-05-02 13:23:45,110 [INFO] Created log file
2024-05-02 13:23:45,120 [INFO] Batch processing nodes for net/0.
2024-05-02 13:23:45,128 [INFO] Batch processing nodes for virt_exc/0.
2024-05-02 13:23:45,149 [INFO] Setting up output directory
2024-05-02 13:23:45,150 [INFO] Building cells.
2024-05-02 13:23:45,158 [INFO] Building recurrent connections
2024-05-02 13:23:45,176 [INFO] Network created.
2024-05-02 13:23:45,178 [INFO] Build virtual cell stimulations for thalamus_spikes
2024-05-02 13:23:45,206 [INFO] Starting Simulation
2024-05-02 13:23:49,697 [INFO] Simulation finished, finalizing results.
2024-05-02 13:23:51,325 [INFO] Done.


One thing to note is that when you replace one cell-model with another you may often get widely varying results. A network that is stable or silent in using one cell type may explode when a model gets replaced. Fixing this may require the or both of the following following:
* Adjusting the **dynamics_params** of the new cell type.
* Adjust the synaptic models and parameters (especially **syn_weight**).

## Example: Importing Custom NESTML cell models